In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from lxml import etree

In [52]:
#-------------------Extract-------------------------

url = "https://forecast.weather.gov/MapClick.php?lat=41.884250000000065&lon=-87.63244999999995#.Y-q4ltLMJkj"
r = requests.get(url)
soup = BeautifulSoup(r.content,"html.parser")

#various containers
item1 = soup.find_all(id='current_conditions-summary')
item2 = soup.find_all(id='current_conditions_detail')

#raw data
temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
temp_c = [item.find(class_="myforecast-current-sm").get_text() for item in item1]

#df of temperatures
df_temp = pd.DataFrame({"Temperature_F" : temp_f, "Temperature_C": temp_c})

#df_2 is a df of current conditions in detail
table = soup.find_all('table')
df_2 = pd.read_html(str(table))[0]
df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

#merge both dataframes
source_df=pd.concat([df_temp,df_2],axis=1)

#scrape lattitude, longitude, and elevation 
lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

#scrape name
location = soup.find('h2', {'class': 'panel-title'}).text.strip()

#add location, lat, long, and elev to source_df
source_df['elevation_ft'] = elev
source_df['latitude'] = lat
source_df['longitude'] = lon
source_df['location'] = location


display(source_df)



,Temperature_F,Temperature_C,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Chill,Wind Speed,elevation_ft,latitude,longitude,location
0,53°F,12°C,29.71 in (1006.4 mb),33°F (1°C),47%,14 Feb 12:53 pm CST,10.00 mi,49°F (9°C),S 14 G 26 mph,617,41.78,87.76,"Chicago, Chicago Midway Airport (KMDW)"


In [59]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By

URLS = ["https://www.wunderground.com/history/daily/us/or/portland", 'https://www.wunderground.com/history/daily/us/ak/sitka/PASI']
for URL in URLS:

    driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
    driver.get(URL)
    sleep(10)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text
    print(precip)

    driver.quit()

/home/reed/Desktop/epicodus/team-week3/reed-work/venvteamweek/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == "__main__":


0.14
0.11


In [ ]:
#-------------Transformations------------------------


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

with open('cities.json') as f:
    cities = json.load(f)

for city in cities:
    response = requests.get(city['NWS_URL'])
    soup = BeautifulSoup(response.content, 'html.parser')

    location = soup.find('h2', {'class': 'panel-title'})
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)
    temperature = soup.find('p', {'class': 'myforecast-current-lrg'})
    humidity_elem = soup.find('td', text='Humidity')
    humidity = humidity_elem.find_next('td').text.strip() if humidity_elem else ''
    wind_speed_elem = soup.find('td', text='Wind Speed')
    wind_speed = wind_speed_elem.find_next('td').text.strip() if wind_speed_elem else ''
    barometer_elem = soup.find('td', text='Barometer')
    barometer = barometer_elem.find_next('td').text.strip() if barometer_elem else ''
    dewpoint_elem = soup.find('td', text='Dewpoint')
    dewpoint = dewpoint_elem.find_next('td').text.strip() if dewpoint_elem else ''
    visibility_elem = soup.find('td', text='Visibility')
    visibility = visibility_elem.find_next('td').text.strip() if visibility_elem else ''
    wind_chill_elem = soup.find('td', text='Wind Chill')
    wind_chill = wind_chill_elem.find_next('td').text.strip() if wind_chill_elem else ''
    last_update_elem = soup.find('td', text='Last update')
    last_update = last_update_elem.find_next('td').text.strip() if last_update_elem else ''

    print({
        'Name': city['Name'],
        'Lat': lat + '°N',
        'Lon': lon + '°W',
        'Elev': elev + 'ft.',
        'Temperature': temperature.text if temperature else '',
        'Humidity': humidity,
        'Wind Speed': wind_speed,
        'Barometer': barometer,
        'Dewpoint': dewpoint,
        'Visibility': visibility,
        'Wind Chill': wind_chill,
        'Last update': last_update
    })